In [1]:
"""
Purpose:
--------
Demonstrate how to query the h01 c2 datajoint data products
using the modularized API class
"""

'\nPurpose:\n--------\nDemonstrate how to query the h01 c2 datajoint data products\nusing the modularized API class\n'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import connects_neuvue

# Step 1: Import API and configure API object with aws secret

In [4]:
from connects_neuvue.utils import aws_utils as aws
secret_dict = aws.get_secret()

In [5]:
from connects_neuvue.h01_c2 import api
fetcher = api.API(secret_dict=secret_dict)
fetcher

/usr/local/lib/python3.8/dist-packages/python_jsonschema_objects/__init__.py:113: UserWarning: Schema id not specified. Defaulting to 'self'
  warnings.warn("Schema id not specified. Defaulting to 'self'")
[2025-05-09 19:59:15,793][INFO]: Connecting admin@neurd-datajoint.cluster-cjc6cqmcqirl.us-east-1.rds.amazonaws.com:3306
INFO - 2025-05-09 19:59:15,793 - connection - Connecting admin@neurd-datajoint.cluster-cjc6cqmcqirl.us-east-1.rds.amazonaws.com:3306
[2025-05-09 19:59:16,410][INFO]: Connected admin@neurd-datajoint.cluster-cjc6cqmcqirl.us-east-1.rds.amazonaws.com:3306
INFO - 2025-05-09 19:59:16,410 - connection - Connected admin@neurd-datajoint.cluster-cjc6cqmcqirl.us-east-1.rds.amazonaws.com:3306


# Application 0: Fetch soma features

In [9]:
import numpy as np

In [10]:
segment_id = 28059648040
key = dict(segment_id=segment_id)

In [11]:
soma_center = fetcher.fetch_soma_center(segment_id)
soma_center

array([307623,  74465,   5038])

In [12]:
soma_center_nm = soma_center*np.array([8,8,33])
soma_center_nm

array([2460984,  595720,  166254])

In [13]:
mesh = fetcher.fetch_segment_id_mesh(**key)

# Application 1: Fetch neuron branch level nx object (and proofread skeleton/mesh for checking)

In [14]:
decimated_mesh = mesh

In [15]:
G = fetcher.nx_graph_autoproof_from_segment_id(segment_id) 
G

In [16]:
# fetch proofread mesh and skeleton to spot check
proofread_mesh = fetcher.fetch_proofread_mesh(
    original_mesh = decimated_mesh,
    **key)
proofread_mesh

proofread_skeleton = fetcher.fetch_proofread_skeleton(
    **key)
proofread_skeleton

array([[[2344241.48962724,  634513.57222932,   52854.01347651],
        [2344238.96838124,  634585.39013966,   52923.55207096]],

       [[2344238.96838124,  634585.39013966,   52923.55207096],
        [2344235.9268238 ,  634657.9428046 ,   52992.30150115]],

       [[2344235.9268238 ,  634657.9428046 ,   52992.30150115],
        [2344232.61253489,  634731.2233965 ,   53060.26338527]],

       ...,

       [[2460991.33907114,  595720.73449779,  166270.89436701],
        [2467125.65441176,  588096.49264706,  170861.40073529]],

       [[2460991.33907114,  595720.73449779,  166270.89436701],
        [2447940.        ,  591132.        ,  162541.        ]],

       [[2460991.33907114,  595720.73449779,  166270.89436701],
        [2465780.        ,  591541.6       ,  173977.3       ]]])

# Unalignment of skeleton vertices

## Utiity functions

In [17]:
import numpy as np

In [18]:
##############################################################################################################
# Linear Algebra Utility Functions
##############################################################################################################
def projection(
    vector_to_project,
    line_of_projection,
    idx_for_projection=None,
    verbose = False,
    return_magnitude = False):
    
    """
    Purpose: Will find the projection of a vector onto a line
    """
    
    line_of_projection = np.array(line_of_projection)
    vector_to_project = np.array(vector_to_project)
    
    if idx_for_projection is not None:
        vector_to_project = vector_to_project[idx_for_projection]
        line_of_projection = line_of_projection[idx_for_projection]
        
    #2) Find the magnitude of projection of new point onto upward middle vector non scaled
    magn = (vector_to_project@line_of_projection)/(
            line_of_projection@line_of_projection)
    proj_v = magn*line_of_projection
    
    if verbose:
        print(f"magn = {magn}")
        print(f"proj_v = {proj_v}")
        
    if return_magnitude:
        return proj_v,magn
    else:
        return proj_v
    
def error_from_projection(
    vector_to_project,
    line_of_projection,
    idx_for_projection=None,
    verbose = False,):
    
    """
    Purpose: To return the error vector of a projection
    
    Ex: 
    
    error_from_projection(
    vector_to_project=orienting_coords["top_left"],
    line_of_projection = hu.upward_vector_middle_non_scaled,
    verbose = True,
    idx_for_projection=np.arange(0,2)
    )
    """
    
    proj_v = projection(
    vector_to_project=vector_to_project,
    line_of_projection=line_of_projection,
    idx_for_projection=idx_for_projection,
    verbose = verbose,
    return_magnitude = False)
    
    if idx_for_projection is not None:
        error_proj = vector_to_project[idx_for_projection] - proj_v
    else:
        error_proj = vector_to_project - proj_v
    
    if verbose:
        print(f"error_proj = {error_proj}")
        
    return error_proj

def perpendicular_vec_2D(vec):
    return np.array([vec[1],-vec[0]])

def rotation_matrix_2D(angle):
    theta = np.radians(angle)
    r = np.array(( (np.cos(theta), -np.sin(theta)),
               (np.sin(theta),  np.cos(theta)) ))
    return r

In [19]:
##############################################################################################################
# H01 Utility Functions
##############################################################################################################

voxel_to_nm_scaling = np.array([8,8,33])
source = "h01"
current_nucleus_version = 0
# -------------- Functions for finding the rotation matrix needed -------
top_radius =700_000 #when magn = 1
bottom_radius = 350_000 #when magn = 0
max_rotation_global = -30
upward_vector_middle = np.array([ 0.94034618, -0.34021915,  0.        ])
upward_vector_middle_non_scaled = np.array([1808188.88892619, -654206.39541785,       0.        ])
upward_vector_start_point = np.array([1041738.17659344, 1785911.29763922,  125032.57443884])
align_vector = np.array([ 0.85082648, -0.52544676,  0.        ])
upward_vector_top_left = np.array([])

verbose = True


def radius_for_rotation_from_proj_magn(magn):
    return (top_radius-bottom_radius)*magn + bottom_radius

def rotation_from_proj_error_and_radius(
    proj_error,
    radius_for_rotation,
    max_rotation = max_rotation_global,
    verbose = False
    ):
    """
    Purpose: To calculate the amount of rotation necessary 
    based on the current radius of rotation and error
    magnitude of the projection
    
    """
    magn_err = np.linalg.norm(proj_error)
    if verbose:
        print(f"magn_err = {magn_err}")
        
    rotation = max_rotation*(magn_err/radius_for_rotation)
    
    if verbose:
        print(f"rotation = {rotation} (max_rotation = {max_rotation})")
        
    return rotation
    
def rotation_signed_from_middle_vector(
    coordinate,
    origin_coordinate=upward_vector_start_point,
    middle_vector = upward_vector_middle_non_scaled,
    zero_out_z_coord = True,
    verbose = False,
    ):
    """
    Purpose: Determine the direction
    and amount of rotation needed for a 
    neuron based on the location of the soma

    Pseudocode: 
    1) Compute the new relative vector to starting vector
    2) Find the magnitude of projection of new point onto upward middle vector non scaled
    3) Use the magnitude of the projection to find the slope of the rotation function
    4) Find the error distance between point and projection distance
    5) Determine the amount of rotation needed based on radius and error projection magnitude
    6) Determine the sign of the rotation
    
    Ex: 
    rotation_signed_from_middle_vector(
    coordinate = orienting_coords["bottom_far_right"],
        verbose = True
    )
    """
    #print(f"verbose rotation_signed_from_middle_vector = {verbose}")
    #1) Compute the new relative vector to starting vector
    v = coordinate - origin_coordinate
    m = middle_vector

    
    if zero_out_z_coord:
        idx_for_projection = np.arange(0,2)
    else:
        idx_for_projection = np.arange(0,3)

    if verbose:
        print(f"new vector = {v}")

    #2) Find the magnitude of projection of new point onto upward middle vector non scaled
    proj_v,magn = projection(
        vector_to_project=v,
        line_of_projection=m,
        idx_for_projection=idx_for_projection,
        verbose=verbose,
        return_magnitude=True
    )

    #3) Use the magnitude of the projection to find the slope of the rotation function
    radius_rot = radius_for_rotation_from_proj_magn(magn)
    if verbose:
        print(f"radius_rot= {radius_rot}")

    #4) Find the error distance between point and projection distance
    proj_er = error_from_projection(
        vector_to_project=v,
        line_of_projection=m,
        idx_for_projection=idx_for_projection,
        verbose=False
    )

    #5) Determine the amount of rotation needed based on radius and error projection magnitude
    curr_rotation = rotation_from_proj_error_and_radius(
        proj_error = proj_er,
        radius_for_rotation = radius_rot,
        verbose = False
    )

    #6) Determine the sign of the rotation
    if zero_out_z_coord:
        m_perp = perpendicular_vec_2D(m[idx_for_projection])
        rotation_sign = np.sign(m_perp @ proj_er)
    else:
        rotation_sign = None

    if verbose:
        print(f"rotation_sign = {rotation_sign}")
        print(f"curr_rotation = {curr_rotation}")
    
    return curr_rotation*rotation_sign

def rotation_from_soma_center(soma_center,
                             verbose = False,
                             **kwargs):
    """
    Purpose: To get the amout r tation necessary from soma
    center of neuron
    
    """
    soma_center = np.array(soma_center).reshape(-1)
    rotation = rotation_signed_from_middle_vector(soma_center,
                                                     verbose = verbose,
                                                     **kwargs)
    if verbose:
        print(f"rotation = {rotation}")
    return rotation

def aligning_matrix_3D_linalg(starting_vector,target_vector):
    """
    Derivation from here: 
    https://math.stackexchange.com/questions/180418/calculate-rotation-matrix-to-align-vector-a-to-vector-b-in-3d

    
    Will create a matrix that will move the vector starting_vector to be aligned with vector target_vector
    """
    upward_vector = starting_vector
    microns_top_vector = target_vector


    c = microns_top_vector @ upward_vector
    v = np.cross(upward_vector,microns_top_vector)
    
    vp = np.array([[0,-v[2],v[1]],[v[2],0,-v[0]],[-v[1],v[0],0]])

    R = np.eye(3,3) + vp + (vp @ vp)*(1 /(1-c))
    R[:2,:2] = R[:2,:2]*-1
    return R

top_of_layer_vector =  np.array([0,-1,0])
def aligning_matrix_3D(upward_vector=align_vector,#upward_vector_middle,
                target_vector = top_of_layer_vector,
                       rotation = None,
                       verbose = False,
                      ):
    """
    Will come up with an alignment matrix
    """
    upward_vector_new = upward_vector.copy()
    
    if rotation is not None:
        if verbose:
            print(f"upward_vector_new before = {upward_vector_new}")
        upward_vector_new[:2] = rotation_matrix_2D(rotation) @ upward_vector_new[:2]
        if verbose:
            print(f"upward_vector_new AFTER = {upward_vector_new}")
    return aligning_matrix_3D_linalg(upward_vector_new,target_vector)

def align_matrix_from_rotation(upward_vector=None,
                              rotation=None,
                              **kwargs):
    if upward_vector is not None:
        kwargs["upward_vector"] = upward_vector
    if rotation is not None:
        kwargs["rotation"] = rotation
    return aligning_matrix_3D(**kwargs)

def align_array(
    array,
    soma_center=None,
    rotation = None,
    align_matrix = None,
    verbose = False,
    **kwargs
    ):
    #print(f"verbose align_array_from_soma_coordinate = {verbose}")
    """
    Purpose: Will align a coordinate or skeleton
    (or any array) with the rotation matrix
    determined from the soam center
    """
    if len(array) <= 0:
        return array
    
    
    curr_shape = array.shape
    #verbose = True
    
    if align_matrix is None:
        if rotation is None:
            rotation = rotation_from_soma_center(soma_center,
                                     verbose = False,
                                     **kwargs)
            if verbose:
                print(f"rotation = {rotation}")
        
    
        align_matrix = align_matrix_from_rotation(
            rotation = rotation,
            verbose = verbose,
        )
        
        if verbose:
            print(f"align_matrix = {align_matrix}")

    new_coords = array.reshape(-1,3) @ align_matrix
    new_array = new_coords.reshape(*curr_shape)
    
    return new_array

def unalign_array(array, align_matrix):
    inv_matrix = np.linalg.inv(align_matrix)
    curr_shape = array.shape

    new_coords = array.reshape(-1,3) @ inv_matrix
    return new_coords.reshape(*curr_shape)

                                      

# rotation=rotation_from_soma_center(
#                 soma_center = soma_center,
#                 verbose = verbose,
# )
# align_matrix = align_matrix_from_rotation(rotation = rotation)

class AlignerH01:
    def __init__(self,soma_center=None,segment_id = None):
        if soma_center is None:
            soma_center = fetcher.fetch_soma_center(segment_id)
        soma_center_nm = soma_center*voxel_to_nm_scaling
        self.soma_center = soma_center
        self.soma_center_nm = soma_center_nm
        self._compute_alignment_matrix()

    def _compute_alignment_matrix(self):
        self.rotation=rotation_from_soma_center(
                soma_center = self.soma_center_nm,
                verbose = False,
        )
        self.align_matrix = align_matrix_from_rotation(rotation = self.rotation)
        self.unalign_matrix = np.linalg.inv(self.align_matrix)

    def _apply_matrix_to_array(self,array,matrix):
        curr_shape = array.shape
        new_coords = array.reshape(-1,3) @ matrix
        return new_coords.reshape(*curr_shape)

    def align_array(self,array):
        return self._apply_matrix_to_array(array,matrix = self.align_matrix)
    def unalign_array(self,array):
        return self._apply_matrix_to_array(array,matrix = self.unalign_matrix)
        


# Unaligning skeleton Vertices

In [20]:
sk_points = G.nodes['L0_7']['skeleton_data']
sk_points

array([[2442864.46741534, -688917.62211729,  135389.75607715],
       [2443217.91079519, -688862.83280099,  134460.31504024],
       [2443475.14031457, -688653.91877222,  133517.04662183],
       [2443650.39090776, -688518.42651917,  132543.40996302],
       [2443790.95455664, -688470.99752306,  131554.54063436],
       [2443968.00682248, -688364.31094322,  130576.36727601],
       [2444063.21786902, -688279.44970699,  129587.13772096],
       [2443917.57518699, -688179.63176012,  128604.66684484],
       [2443918.29967137, -687959.15478577,  127632.52622257],
       [2444059.45734671, -687819.83850892,  126653.26451019],
       [2444192.0205802 , -687802.42882613,  125662.78888545],
       [2444243.90316643, -687843.00780086,  124665.4470035 ],
       [2444197.31076117, -687909.73686649,  123668.97518263],
       [2444170.09466071, -687917.67050855,  122670.07437749],
       [2444214.4353558 , -687815.47443911,  121676.94630648],
       [2444259.25558626, -687653.88737341,  120691.370

In [21]:
align_obj = AlignerH01(segment_id=segment_id)
align_obj

In [22]:
sk_points_unaligned = align_obj.unalign_array(sk_points)
sk_points_unaligned

array([[2463657.97799424,  610396.56796374,  135389.75607715],
       [2463937.98214483,  610619.10284569,  134460.31504024],
       [2464058.10291689,  610927.94439471,  133517.04662183],
       [2464143.32699318,  611132.41390395,  132543.40996302],
       [2464241.99455762,  611243.19459644,  131554.54063436],
       [2464343.04069211,  611423.52543655,  130576.36727601],
       [2464383.77366195,  611544.38650861,  129587.13772096],
       [2464207.81721259,  611559.0406123 ,  128604.66684484],
       [2464099.32818891,  611750.98006929,  127632.52622257],
       [2464153.03503448,  611941.89913831,  126653.26451019],
       [2464259.60790235,  612022.63532978,  125662.78888545],
       [2464324.77403349,  612013.05245423,  124665.4470035 ],
       [2464317.31368341,  611932.00947848,  123668.97518263],
       [2464297.59109183,  611911.64582898,  122670.07437749],
       [2464285.54150034,  612022.39302663,  121676.94630648],
       [2464244.51477893,  612184.98466949,  120691.370

In [25]:
align_obj.align_matrix

array([[ 0.86893829, -0.49492039,  0.        ],
       [ 0.49492039,  0.86893829,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [23]:
"""
correct alignment matrix (pulled from neuron object)
------------------------
array([[ 0.8689366 , -0.49492336,  0.        ],
       [ 0.49492336,  0.8689366 ,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])
"""

'\ncorrect alignment matrix (pulled from neuron object)\n------------------------\narray([[ 0.8689366 , -0.49492336,  0.        ],\n       [ 0.49492336,  0.8689366 ,  0.        ],\n       [ 0.        ,  0.        ,  1.        ]])\n'

# Plotting to verify correct unalignment

In [24]:
try:
    from datasci_tools import ipyvolume_utils as ipvu
    ipvu.plot_objects(
        main_skeleton=proofread_skeleton,
        scatters=[sk_points_unaligned]
    )
except:
    print(f"Need datasci_stdlib_tools installed to plot")

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…